## *Libraries*

In [ ]:
# Libraries
import numpy as np
import random
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.activations import relu, softmax

# Used to prepare the data
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Set random seeds
# for consistent results, 
# If you remove the next lines, the results will not be reproducible 
SEED = 42
np.random.seed(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)

# Ensure deterministic operations (only for GPU execution)
os.environ['TF_DETERMINISTIC_OPS'] = '1'

# Python version in the environment (to use tensorflow it should be 3.11.* or lower)
# I used 3.11.9
import sys
print(f"Using Python version: {sys.version}")

# TensorFlow version
print(f"Using TensorFlow version: {tf.__version__}")

Using Python version: 3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)]
Using TensorFlow version: 2.19.0


## *Loading and splitting the data* 

In [2]:
# Loading the data
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Combine training and test data for reshuffling
x_original = np.concatenate((x_train, x_test), axis=0)
y_original = np.concatenate((y_train, y_test), axis=0)

print("x shape:", x_original.shape)
print("y shape:", y_original.shape)

x shape: (70000, 28, 28)
y shape: (70000,)


*Devide the data into 3 sets (training, cross validation and test) to compare each model performance later*

In [3]:
# Get 60% of the dataset as the training set. 
# Put the remaining 40% in temporary variables: x_ and y_.

x_train, x_, y_train, y_ = train_test_split(x_original, y_original, test_size = 0.3, random_state=1)

# Split the 40% subset above into two: 
# one half for cross validation and the other for the test set

x_dev, x_test, y_dev, y_test = train_test_split(x_, y_, test_size = 0.33, random_state=1)

del x_, y_

print(f"the shape of the training set (input) is: {x_train.shape}")
print(f"the shape of the training set (target) is: {y_train.shape}\n")
print(f"the shape of the cross validation set (input) is: {x_dev.shape}")
print(f"the shape of the cross validation set (target) is: {y_dev.shape}\n")
print(f"the shape of the test set (input) is: {x_test.shape}")
print(f"the shape of the test set (target) is: {y_test.shape}")

the shape of the training set (input) is: (49000, 28, 28)
the shape of the training set (target) is: (49000,)

the shape of the cross validation set (input) is: (14070, 28, 28)
the shape of the cross validation set (target) is: (14070,)

the shape of the test set (input) is: (6930, 28, 28)
the shape of the test set (target) is: (6930,)


*Normalize the data to improve performance and avoid biased weights*

In [4]:
# Normalize the data
x_train = x_train / 255.0
x_dev = x_dev / 255.0
x_test = x_test / 255.0

# Ensure CNN compatibility
x_train_cnn = x_train.reshape(-1, 28, 28, 1)
x_dev_cnn = x_dev.reshape(-1, 28, 28, 1)
x_test_cnn = x_test.reshape(-1, 28, 28, 1)

## *Models implementation and training*

In [6]:
# MODELS IMPLEMENTATION

models = [
    Sequential([Flatten(input_shape=(28, 28)), Dense(10, activation='softmax')], name='Baseline_Model'),
    Sequential([Flatten(input_shape=(28, 28)), Dense(64, activation='relu'), Dense(10, activation='softmax')], name='Small_Model'),
    Sequential([Flatten(input_shape=(28, 28)), Dense(32, activation='relu'), Dense(16, activation='relu'), Dense(10, activation='softmax')], name='Medium_Model'),
    Sequential([
        Conv2D(16, (3,3), activation='relu', input_shape=(28, 28, 1)),
        MaxPooling2D(pool_size=(2,2)),
        Flatten(),
        Dense(32, activation='relu'),
        Dense(10, activation='softmax')
    ], name='CNN_Model')
]

In [7]:
# Compiling all models
for model in models:

    # Setup the loss and optimizer
    model.compile(
        loss = tf.keras.losses.SparseCategoricalCrossentropy(),
        optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001)
    )

    print(f"Training {model.name}...")

    # Train the model
    x_train_input = x_train_cnn if 'CNN' in model.name else x_train
    model.fit(
        x_train_input, y_train,
        epochs = 100,
        verbose = 0   # Silent mode, no output during training.
    )
    
    print("Done!\n")

Training Baseline_Model...
Done!

Training Small_Model...
Done!

Training Medium_Model...
Done!

Training CNN_Model...
Done!



## *Evaluating the models* 

*First compare the training error with the cross validation error and choose the one with the smallest cross validation error but also  avoiding overfitting in case the training error is to small comparing with the cross validation error*

In [8]:
# Initialize lists that will contain the errors for each model
nn_train_errors = []
nn_cv_errors = []

# Calculate the errors
for model in models:
    x_train_input = x_train_cnn if 'CNN' in model.name else x_train
    x_dev_input = x_dev_cnn if 'CNN' in model.name else x_dev

    # Calculate the training error and store it
    train_predictions = model.predict(x_train_input)
    train_accuracy = accuracy_score(y_train, train_predictions.argmax(axis=1))
    nn_train_errors.append(1 - train_accuracy)

    # Calculate the cross-validation error and store it
    dev_predictions = model.predict(x_dev_input)
    dev_accuracy = accuracy_score(y_dev, dev_predictions.argmax(axis=1))
    nn_cv_errors.append(1 - dev_accuracy)

# Comparing the errors for all models
print("\nRESULTS:")

for idx, (train_error, dev_error) in enumerate(zip(nn_train_errors, nn_cv_errors), 1):
    print(f"Model {idx}: Training Error: {train_error:.2%}, Cross validation Error: {dev_error:.2%}")

1532/1532 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step
440/440 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
440/440 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step
440/440 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
440/440 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step

RESULTS:
Model 1: Training Error: 5.62%, Cross validation Error: 8.22%
Model 2: Training Error: 0.02%, Cross validation Error: 2.73%
Model 3: Training Error: 0.69%, Cross validation Error: 4.02%
Model 4: Training Error: 0.00%, Cross validation Error: 1.53%


*Generally perfect training error indicates an issue of overfitting.*

## *Chosing a model*

*For illustrative purposes in this notebook we test all the models with the test set, but in practice this can be done only with the chossed model based on performance*

In [9]:
for model in models:

    x_test_input = x_test_cnn if 'CNN' in model.name else x_test

    # Predict class probabilities for the training set using models
    predictions = model.predict(x_test_input)

    # Get the predicted class labels
    yhat = np.argmax(predictions, axis=1)

    # Misclassified data
    misclassified = sum(yhat != y_test)

    # Compute the fraction of the data that the model misclassified
    fraction_error = misclassified / len(predictions)
    
    print(f"fraction of misclassified data for {model.name}: {fraction_error}")

217/217 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
fraction of misclassified data for Baseline_Model: 0.08282828282828283
217/217 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
fraction of misclassified data for Small_Model: 0.02886002886002886
217/217 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
fraction of misclassified data for Medium_Model: 0.04040404040404041
217/217 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
fraction of misclassified data for CNN_Model: 0.016305916305916306


*We can now save the model with the best performance on the test set*

In [11]:
# Saving the best model based on the performance over the test set
# uncomment the line below to save the model

models[1].save('M1_digit_rec.keras')
models[3].save('M2_digit_rec.keras')